In [81]:
## Target implement RAG
# 1) chunking docs
# 2) persistent embeddings
# 3) query vector store
# 4) compose a response based on documents

In [82]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [83]:
base_url="http://localhost:11434"
model="zephyr"  #orca-mini , mistral, or zephyr

llm = Ollama(base_url=base_url, model=model, 
callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

In [84]:
#llm("Hola!")
llm("Draft welcome talking points for the 'TechNovation Talk: Supercharging Innovation Acceleratio' with Featured Speaker Dr. Adler Archer: Managing Director, Inclusive Innovation Initiative (i3), and former Director of the Office of Strategy Management at the Whiting School of Engineering, Johns Hopkins University. This event is the first of a series organized in collaboration with Johns hopkins University and UN-OICT. This event is put together through a collaboration of the OICT Innovation Unit, the OICT Open Source Team and the DMSPC/BTAD Innovation Accelerator.  The TechNovation Talks are a series of learning and networking opportunities that bring together external experts and UN personnel to acquire skills essential for building the UN 2.0.")

Dear participants,

We are thrilled to welcome Dr. Adler Archer, Managing Director of the Inclusive Innovation Initiative (i3) and former Director of the Office of Strategy Management at the Whiting School of Engineering, Johns Hopkins University, as our featured speaker for tonight's TechNovation Talk: Supercharging Innovation Acceleration.

This event is part of a collaborative effort between the United Nations Office of Information and Communications Technology (OICT), the OICT Open Source Team, and the DMSPC/BTAD Innovation Accelerator, aimed at enhancing innovation within the UN system. Our TechNovation Talks are designed to provide learning and networking opportunities for UN personnel to acquire essential skills necessary for building the UN 2.0.

Dr. Archer brings a wealth of experience in driving innovation, particularly in the realm of inclusive innovation. His work with i3 has focused on creating social and economic value through innovation while simultaneously addressing gl

"Dear participants,\n\nWe are thrilled to welcome Dr. Adler Archer, Managing Director of the Inclusive Innovation Initiative (i3) and former Director of the Office of Strategy Management at the Whiting School of Engineering, Johns Hopkins University, as our featured speaker for tonight's TechNovation Talk: Supercharging Innovation Acceleration.\n\nThis event is part of a collaborative effort between the United Nations Office of Information and Communications Technology (OICT), the OICT Open Source Team, and the DMSPC/BTAD Innovation Accelerator, aimed at enhancing innovation within the UN system. Our TechNovation Talks are designed to provide learning and networking opportunities for UN personnel to acquire essential skills necessary for building the UN 2.0.\n\nDr. Archer brings a wealth of experience in driving innovation, particularly in the realm of inclusive innovation. His work with i3 has focused on creating social and economic value through innovation while simultaneously addres

In [85]:
#Directory loader, including text, pdf
from langchain.document_loaders import DirectoryLoader
input_dir = "./data_big/"
data = DirectoryLoader(input_dir , use_multithreading=True).load()
len(data)

Invalid dictionary construct: [/'CS', <PDFObjRef:6926>, /'I', False, /'K', /b'fals', /b'e', /'S', /'Transparency', /'Type', /'Group']
Traceback (most recent call last):
  File "/Users/jmn/repos/AiFossStack/.venvlangllama/lib/python3.11/site-packages/unstructured/partition/pdf.py", line 251, in partition_pdf_or_image
    extracted_elements = extractable_elements(
                         ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jmn/repos/AiFossStack/.venvlangllama/lib/python3.11/site-packages/unstructured/partition/pdf.py", line 196, in extractable_elements
    return _partition_pdf_with_pdfminer(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jmn/repos/AiFossStack/.venvlangllama/lib/python3.11/site-packages/unstructured/utils.py", line 214, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jmn/repos/AiFossStack/.venvlangllama/lib/python3.11/site-packages/unstructured/partition/pdf.py", line 489, in _partition_pdf_with_pdfminer
    elements

7

In [86]:
# split it into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)
len(all_splits)

2367

In [87]:
#Vectorize & store
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
index_name = "undocs"
vectorstore = Chroma.from_documents(documents=all_splits, embedding=GPT4AllEmbeddings(), persist_directory="./indexes/" + index_name)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [101]:
# load chromadb vectorstore from disk <-works
index_name = "undocs"
db2 = Chroma(persist_directory="./indexes/" + index_name , embedding_function=GPT4AllEmbeddings())
print("Rows:", db2._collection.count())

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
Rows: 4734


In [98]:
# load from disk & query <-works
index_name = "long"
db3 = Chroma(persist_directory="./indexes/" + index_name, embedding_function=GPT4AllEmbeddings())
query = "umoja"
docs = db3.similarity_search(query)
docs

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


[Document(page_content='measures, national and international, to secure their universal and effective recognition and observance, both among the peoples of Member States themselves and among the peoples of territories under their jurisdiction.', metadata={'source': 'data/Universal Declaration of Human Right.txt'}),
 Document(page_content='women and have determined to promote social progress and better standards of life in larger freedom, Whereas Member States have pledged themselves to achieve, in co‐operation with the United Nations, the promotion of universal respect for and observance of human rights and fundamental freedoms, Whereas a common understanding of these rights and freedoms is of the greatest importance for the full realization of this pledge, Now, therefore, The General Assembly Proclaims this Universal Declaration of Human Rights as a common standard of achievement for all peoples and all nations, to the end that every individual and every organ of society, keeping this

In [104]:
# search with score <--- works
query = "what is resident coordinator?"

docs = db2.similarity_search_with_score(query) #the score is the last value of the metadte for each doc 
#lower the score the more similar. 

docs
#print(docs[0].page_content)
#print(docs[0].metadata)
#docs[0]

#len(docs) #by default, the top 4 results are returned


[(Document(page_content='The Board recommended that the Administration develop a resident coordinator handbook to provide more guidance to resident coordinators ad interim in delivering the expected results.\n\nThe Board recommended that the Administration intensify efforts to expand the resident coordinator pool to include more viable\n\nAdministration’s response', metadata={'source': 'data_big/A:78:5 (Vol.I)2022.pdf'}),
  0.6247134208679199),
 (Document(page_content='The Board recommended that the Administration develop a resident coordinator handbook to provide more guidance to resident coordinators ad interim in delivering the expected results.\n\nThe Board recommended that the Administration intensify efforts to expand the resident coordinator pool to include more viable\n\nAdministration’s response', metadata={'source': 'data_big/A:78:5 (Vol.I)2022.pdf'}),
  0.6247134208679199),
 (Document(page_content='492. The Administration accepted the recommendation.\n\n(c) Concentrated geog

In [ ]:
# delete collection
#print("Count:", db2._collection.count())
#db2.delete_collection()